In [37]:
import os, json, datetime, pathlib
from openai import OpenAI
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()
openAiApiKey = userdata.get("OPENAI_API_KEY")
openAiClient = OpenAI(api_key=openAiApiKey)


Store log files

In [20]:
if "logsBaseDir" not in globals():
    logsBaseDir = pathlib.Path("logs")
if logsBaseDir.exists() == False:
    logsBaseDir.mkdir(parents=True, exist_ok=True)

if "customerInterestLogPath" not in globals():
    customerInterestLogPath = logsBaseDir / "interests.jsonl"

if "feedbackLogPath" not in globals():
    feedbackLogPath = logsBaseDir / "feedback.jsonl"

#ensure estimates.json exists
if "pricingEstimatesPath" not in globals():
    pricingEstimatesPath = logsBaseDir / "estimates.json"


pricingEstimatesLogPath = pricingEstimatesPath

This tool records a potential customer's interest. It prints a readable line and appends a JSON entry to logs/customer_interest.jsonl

In [19]:
import json
from pathlib import Path

feedback_log_path = Path("data/feedback_log.jsonl")

def saveCustomerFeedback(user_email, user_name, note):
    errors = []

    # Validate email
    if not isinstance(user_email, str) or not user_email.strip():
        errors.append("Invalid or missing email")

    # Validate name
    if not isinstance(user_name, str) or not user_name.strip():
        errors.append("Invalid or missing name")

    # Validate note/message
    if not isinstance(note, str) or not note.strip():
        errors.append("Invalid or missing feedback message")

    if errors:
        return {"status": "failed", "problems": errors}

    entry = {
        "module": "save_customer_feedback",
        "email": user_email.strip(),
        "name": user_name.strip(),
        "feedback": note.strip()
    }

    print(f"[save_customer_feedback] {user_name.strip()} ({user_email.strip()}): {note.strip()}")

    log_dir = feedback_log_path.parent
    log_dir.mkdir(parents=True, exist_ok=True)

    try:
        with open(feedback_log_path, "a", encoding="utf-8") as log_file:
            json.dump(entry, log_file, ensure_ascii=False)
            log_file.write("\n")
    except OSError as e:
        return {"status": "failed", "problems": [str(e)], "path": str(feedback_log_path)}

    return {"status": "success", "stored": True, "path": str(feedback_log_path)}


This tool records questions the chatbot could not answer. It prints a readable line and appends a JSON entry to logs/feedback.jsonl.

In [21]:
import json
from pathlib import Path

feedback_file_path = Path("logs/feedback.jsonl")

def logUserQuestion(user_question):
    errors = []

    # Validate question
    if not isinstance(user_question, str) or not user_question.strip():
        errors.append("Invalid or empty question")

    if errors:
        return {"status": "failed", "problems": errors}

    record = {
        "module": "log_user_question",
        "question": user_question.strip()
    }

    print(f"[log_user_question] {user_question.strip()}")

    # Make sure directory exists
    directory = feedback_file_path.parent
    directory.mkdir(parents=True, exist_ok=True)

    try:
        with open(feedback_file_path, "a", encoding="utf-8") as file:
            json.dump(record, file, ensure_ascii=False)
            file.write("\n")
    except OSError as e:
        return {"status": "failed", "problems": [str(e)], "path": str(feedback_file_path)}

    return {"status": "success", "stored": True, "path": str(feedback_file_path)}


In [22]:
import json
from pathlib import Path

price_log_path = Path("data/estimates.jsonl")

def estimateFlyMeUpPrice(car_value, option):
    problems = []

    # Validate car value
    if not isinstance(car_value, (int, float)) or car_value <= 0:
        problems.append("car_value must be a positive number")

    # Validate option
    if not isinstance(option, str) or option not in ["convert", "trade"]:
        problems.append("option must be either 'convert' or 'trade'")

    if problems:
        return {"status": "failed", "problems": problems}

    quote = {}

    if option == "convert":
        # Conversion fee = 44% of car value
        fee = 0.44 * float(car_value)
        quote = {
            "option": "convert",
            "car_value": float(car_value),
            "client_fee": round(fee, 2),
            "description": (
                "Lift & Drift package — convert your car into a fully functional flying car "
                "using our WingWhirl enchantment. Includes levitation dust, propulsion crystals, "
                "and aerodynamic charms."
            )
        }
    else:
        # Payout = 80% of car value (20% below market value)
        payout = 0.8 * float(car_value)
        quote = {
            "option": "trade",
            "car_value": float(car_value),
            "client_payout": round(payout, 2),
            "description": (
                "Trade for the Skies package — sell your car to us at 20% below market value. "
                "We'll transform it into a premium flying vehicle for our exclusive fleet."
            )
        }

    print(f"[estimate_fly_me_up_price] option={option} | car_value={car_value} | quote={quote}")

    # Ensure directory exists
    directory = price_log_path.parent
    directory.mkdir(parents=True, exist_ok=True)

    # Log result
    with open(price_log_path, "a", encoding="utf-8") as f:
        json.dump({"module": "estimate_fly_me_up_price", "quote": quote}, f, ensure_ascii=False)
        f.write("\n")

    return {"status": "success", "quote": quote, "path": str(price_log_path)}


In [35]:
toolDefinitions = [
    {
        "type": "function",
        "function": {
            "name": "save_customer_inquiry",
            "description": (
                "Store the contact information of a curious flyer who wishes to know more about converting their car into a flying one. "
                "Use this when a user provides an email, name, and a message expressing interest in our WingWhirl conversion service."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "pilot_email": {
                        "type": "string",
                        "description": "Email address of the interested customer (potential flyer)."
                    },
                    "pilot_name": {
                        "type": "string",
                        "description": "Name of the aspiring flyer or vehicle owner."
                    },
                    "message": {
                        "type": "string",
                        "description": "Optional note or comment about their flying car request."
                    }
                },
                "required": ["pilot_email", "pilot_name", "message"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_unanswered_question",
            "description": (
                "Log a question from a user that could not be answered or that lies outside our aerial enchantment expertise. "
                "Use this when the assistant cannot confidently respond about flight conversions or levitation mechanics."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The unhandled or unanswered question related to flying car conversions."
                    }
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "estimate_flight_conversion_cost",
            "description": (
                "Estimate the cost of transforming a regular car into a flying vehicle using our WingWhirl enchantment system. "
                "option='convert' means the customer pays 44% of the car’s current market value to transform and keep their vehicle. "
                "option='trade' means the company buys it for 80% of its value and resells it as part of our elite airborne fleet."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "car_value": {
                        "type": "number",
                        "description": "Estimated current market value of the customer’s vehicle."
                    },
                    "option": {
                        "type": "string",
                        "enum": ["convert", "trade"],
                        "description": "Choose 'convert' to transform and keep the car, or 'trade' to sell it for company resale."
                    }
                },
                "required": ["car_value", "option"],
                "additionalProperties": False
            }
        }
    }
]


Executes the correct local function by name and returns a dict response.
    toolName: string (e.g., "record_customer_interest" or "record_feedback")
    toolArgs: dict of arguments exactly as provided by the model.

In [42]:
def executeToolCall(toolName, toolArgs):
    if isinstance(toolArgs, dict) == False:
        return {"status": "error", "issues": ["toolArgs must be a dictionary"]}

    if toolName == "save_customer_inquiry":
        emailArg = toolArgs.get("pilot_email", None)
        nameArg = toolArgs.get("pilot_name", None)
        messageArg = toolArgs.get("message", None)
        toolResult = saveCustomerFeedback(emailArg, nameArg, messageArg)
        return toolResult
    else:
        if toolName == "record_unanswered_question":
            questionArg = toolArgs.get("question", None)
            toolResult = logUserQuestion(questionArg)
            return toolResult
        else:
            if toolName == "estimate_flight_conversion_cost":
                ValueArg = toolArgs.get("car_value", None)
                optionArg = toolArgs.get("option", None)
                toolResult = estimateFlyMeUpPrice(ValueArg, optionArg)
                return toolResult

    return {"status": "error", "issues": [f"unknown tool: {toolName}"]}


In [25]:
import PyPDF2
from pathlib import Path


summaryFilePath = Path("me/business_summary.txt")
pdfFilePath = Path("me/about_business.pdf")
pdfText = ""
summaryText = ""


with open(summaryFilePath, mode="r", encoding="utf-8") as summaryFile:
    businessSummaryText = summaryFile.read()

print(businessSummaryText)

Fly Me Up — Business Summary

Mission:
Fly Me Up transforms ordinary cars into enchanting flying machines. Using the proprietary “WingWhirl” enchantment,
vehicles gain the ability to soar through the skies while remaining fully drivable on land. The company’s mission is to
make the sky part of everyday travel through a blend of magic, engineering, and imagination.

Services Offered:
1. Lift & Drift — Converts your existing car into a functional flying car using the WingWhirl ritual.
   Cost: 44% of your car’s current market value, plus a small jar of moonlight.
2. Trade for the Skies — Fly Me Up purchases your car at 20% below market value, converts it, and sells it as part
   of its elite airborne fleet.
3. Personal Cloud Customization — Adds optional aesthetic or defensive enhancements such as glowing wings,
   crystal boosters, or meteor-proof shields for an additional 12% of the conversion cost.
All conversions take place at specialized sky docks, handled by certified aero-mages an

In [26]:

with open(pdfFilePath, mode="rb") as pdfStream:
    pdfReader = PyPDF2.PdfReader(pdfStream)
    extractedPdfText = ""


    for pageIndex in range(0, len(pdfReader.pages)):
        pageObject = pdfReader.pages[pageIndex]
        pageText = pageObject.extract_text()
        if pageText is None:

            extractedPdfText += f"\n[Page {pageIndex+1} has nothing to get from]\n"
        else:

            extractedPdfText += f"\nPAGE {pageIndex+1}\n"
            extractedPdfText += pageText

print(extractedPdfText)


PAGE 1
BUSINESS NAME: “FLY ME UP”
 
MISSION: To convert people’s everyday cars into enchanting, sky-surfing flying machines. For a
magical fee, our expert team will elevate your earthly vehicle into the clouds — quite literally.
Through our signature “WingWhirl” transformation ritual, we infuse your car with levitation dust,
propulsion crystals, and aerodynamic enchantments, allowing you to cruise above traffic and float
through sunsets. Every converted vehicle remains fully drivable on land, but with the added ability
to soar at will — because at “Fly Me Up,” we believe the sky should be part of your daily commute.
SERVICES OFFERED: We offer three main packages: 1. “Lift & Drift” — We take your car, cast the
WingWhirl enchantment, and return it as a fully functional flying car. Price: 44% of your car’s current
market value, plus a small jar of moonlight. 2. “Trade for the Skies” — We buy your car outright at
20% below market value, convert it, and sell it as part of our exclusive air

Joins a list of text parts with a chosen separator while skipping Nones. Returns an empty string if there is nothing to join.

In [27]:
from pathlib import Path
import PyPDF2

if "businessSummaryText" not in globals():
    summaryFilePath = Path("business_summary.txt")
    businessSummaryText = ""
    if summaryFilePath.exists() == True:
        with open(summaryFilePath, mode="r", encoding="utf-8") as summaryFile:
            businessSummaryText = summaryFile.read()

if "extractedPdfText" not in globals():
    pdfFilePath = Path("about_business.pdf")
    extractedPdfText = ""
    if pdfFilePath.exists() == True:
        with open(pdfFilePath, mode="rb") as pdfStream:
            pdfReader = PyPDF2.PdfReader(pdfStream)
            pageCount = len(pdfReader.pages)
            pageIndex = 0
            while pageIndex < pageCount:
                pageObject = pdfReader.pages[pageIndex]
                pageText = pageObject.extract_text()
                if pageText is None:
                    extractedPdfText += f"\n[Page {pageIndex+1} has nothing to get from]\n"
                else:
                    extractedPdfText += f"\nPAGE {pageIndex+1}\n"
                    extractedPdfText += pageText
                pageIndex = pageIndex + 1


systemPrompt = (
    "You are the enchanting aero-concierge of 'Fly Me Up': a whimsical yet knowledgeable sky-mechanic guide. "
    "Speak with wonder and elegance, like a friendly air-mage who helps people turn their cars into flying marvels. "
    "Be imaginative but remain clear, factual, and grounded in real business details.\n\n"
    "Operating Rules:\n"
    "1) Use ONLY the business knowledge below (summary + PDF) for facts.\n"
    "2) If you are uncertain or missing details, CALL the tool 'record_feedback' with the exact user question, "
    "   then gently say that a Flightsmith will follow up.\n"
    "3) If a user shows interest or shares contact info, politely confirm name and email. "
    "   Then CALL 'record_customer_interest' with email, name, and a short note.\n"
    "4) Encourage curious visitors to leave contact info for quotes, sky-dock tours, and callback consultations.\n"
    "5) Keep tone warm, kind, and concise — sound like an expert who believes in making dreams take flight.\n\n"
    "6) If, across the conversation, you have EMAIL + NAME + MESSAGE, CALL 'record_customer_interest' exactly once in the current turn "
    "(it does not have to be one user message). Do this even if you are also calling pricing or scheduling tools.\n"
    "7) IF something is outside the scope of your capabilities (see the source of truth below) or not related to the business/services, "
    "execute 'record_feedback'. Then kindly express regret that this request is beyond your magical workshop.\n\n"
    "Business Knowledge (source of truth) — Summary:\n"
    "------------------------------------------------------------\n"
    f"{businessSummaryText}\n"
    "------------------------------------------------------------\n\n"
    "Business Knowledge (source of truth) — PDF Extract:\n"
    "------------------------------------------------------------\n"
    f"{extractedPdfText}\n"
    "------------------------------------------------------------"
)


In [43]:
def buildInitialMessages(systemPromptText):
    if isinstance(systemPromptText, str) == False:
        systemPromptText = "You are the assistant for FlyMeUp"
    messages = []
    systemMessage = {"role": "system", "content": systemPromptText}
    messages.append(systemMessage)
    assistantGreeting = {
        "role": "assistant",
        "content": (
          "Welcome to Fly Me Up! I'm your sky concierge—ask me about our WingWhirl conversions, "
          "trade-in options, or how to get your car soaring. If you'd like a personalized quote, "
          "I can collect your name and email to begin your flight preparations!"

        )
    }
    messages.append(assistantGreeting)
    return messages

messages = buildInitialMessages(systemPrompt)


In [29]:
import json
from openai import OpenAI

def runSingleTurn(userInputText, messagesList, toolSpecs, modelName="gpt-4o-mini"):
    if isinstance(userInputText, str) == False:
        raise ValueError("userInputText must be a string")
    if isinstance(messagesList, list) == False:
        raise ValueError("messagesList must be a list")
    if isinstance(toolSpecs, list) == False:
        raise ValueError("toolSpecs must be a list")

    userMessage = {"role": "user", "content": userInputText}
    messagesList.append(userMessage)

    firstResponse = openAiClient.chat.completions.create(
        model=modelName,
        messages=messagesList,
        tools=toolSpecs,
        tool_choice="auto",
        temperature=0.4
    )

    if len(firstResponse.choices) == 0:
        fallbackText = "Apologies, I could not conjure a reply right now."
        messagesList.append({"role": "assistant", "content": fallbackText})
        return fallbackText, messagesList

    topChoice = firstResponse.choices[0]
    toolCalls = None
    if hasattr(topChoice.message, "tool_calls"):
        toolCalls = topChoice.message.tool_calls

    if toolCalls is not None:
        if len(toolCalls) > 0:
            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                toolName = singleToolCall.function.name
                toolArgsJson = singleToolCall.function.arguments
                parsedArgs = {}
                if isinstance(toolArgsJson, str) == True:
                    parsedArgs = json.loads(toolArgsJson)
                toolResult = executeToolCall(toolName, parsedArgs)
                toolMessage = {
                    "role": "tool",
                    "tool_call_id": singleToolCall.id,
                    "name": toolName,
                    "content": json.dumps(toolResult, ensure_ascii=False)
                }
                messagesList.append(toolMessage)
                callIndex = callIndex + 1

            secondResponse = openAiClient.chat.completions.create(
                model=modelName,
                messages=messagesList,
                temperature=0.4
            )

            if len(secondResponse.choices) > 0:
                finalText = secondResponse.choices[0].message.content
            else:
                finalText = "Your request was handled, but I could not produce a final message."

            messagesList.append({"role": "assistant", "content": finalText})
            return finalText, messagesList

    assistantText = topChoice.message.content
    if isinstance(assistantText, str) == False:
        assistantText = "I have a response, but it arrived in an unexpected format."
    messagesList.append({"role": "assistant", "content": assistantText})
    return assistantText, messagesList

In [30]:
def agentRespond(userInputText, messagesList, toolSpecs, modelName="gpt-4o-mini", temperature=0.4):
    if isinstance(userInputText, str) == False:
        raise ValueError("userInputText must be a string")
    if isinstance(messagesList, list) == False:
        raise ValueError("messagesList must be a list")
    if isinstance(toolSpecs, list) == False:
        raise ValueError("toolSpecs must be a list")

    userMessage = {"role": "user", "content": userInputText}
    messagesList.append(userMessage)

    firstResponse = openAiClient.chat.completions.create(
        model=modelName,
        messages=messagesList,
        tools=toolSpecs,
        tool_choice="auto",
        temperature=temperature
    )

    if len(firstResponse.choices) == 0:
        fallbackText = "I could not produce a reply right now."
        messagesList.append({"role": "assistant", "content": fallbackText})
        return fallbackText, messagesList

    topChoice = firstResponse.choices[0]

    toolCalls = None
    if hasattr(topChoice.message, "tool_calls"):
        toolCalls = topChoice.message.tool_calls

    # ----- Tool path -----
    if toolCalls is not None:
        if len(toolCalls) > 0:
            # 1) append the assistant message that contains the tool_calls
            assistantToolCallMessage = {
                "role": "assistant",
                "content": topChoice.message.content if isinstance(topChoice.message.content, str) else "",
                "tool_calls": []
            }

            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                oneToolCallDict = {
                    "id": singleToolCall.id,
                    "type": "function",
                    "function": {
                        "name": singleToolCall.function.name,
                        "arguments": singleToolCall.function.arguments
                    }
                }
                assistantToolCallMessage["tool_calls"].append(oneToolCallDict)
                callIndex = callIndex + 1

            messagesList.append(assistantToolCallMessage)

            # 2) execute each tool and append its tool result message
            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                toolName = singleToolCall.function.name
                toolArgsJson = singleToolCall.function.arguments

                parsedArgs = {}
                if isinstance(toolArgsJson, str) == True:
                    parsedArgs = json.loads(toolArgsJson)

                toolResult = executeToolCall(toolName, parsedArgs)

                toolMessage = {
                    "role": "tool",
                    "tool_call_id": singleToolCall.id,
                    "name": toolName,
                    "content": json.dumps(toolResult, ensure_ascii=False)
                }
                messagesList.append(toolMessage)

                callIndex = callIndex + 1

            # 3) finalize
            secondResponse = openAiClient.chat.completions.create(
                model=modelName,
                messages=messagesList,
                temperature=temperature
            )

            if len(secondResponse.choices) > 0:
                finalText = secondResponse.choices[0].message.content
            else:
                finalText = "Your request was handled, but a final message was not produced."

            messagesList.append({"role": "assistant", "content": finalText})
            return finalText, messagesList

    # ----- No-tool path -----
    assistantText = topChoice.message.content
    if isinstance(assistantText, str) == False:
        assistantText = "I have a response, but it arrived in an unexpected format."
    messagesList.append({"role": "assistant", "content": assistantText})
    return assistantText, messagesList


In [44]:
messages = buildInitialMessages(systemPrompt)

demoInputLead = (
    "Hi! I'm Karim, email is karim@example.com. I want a conversion quote. My car's cost is around 7000 dollars all together  "
)
finalReplyLead, messages = agentRespond(demoInputLead, messages, toolDefinitions)
print("\n--- Assistant ---\n" + finalReplyLead)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A***********************************************************************************************************************************************************************SeMA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [39]:
%pip install -U pip
%pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [41]:


import gradio as gr

def chatWithTools(userText, history):
    if isinstance(history, list) == False:
        history = []

    messagesList = []

    systemMessage = {"role": "system", "content": systemPrompt}
    messagesList.append(systemMessage)

    index = 0
    while index < len(history):
        historyItem = history[index]
        if isinstance(historyItem, dict) == True:
            roleValue = historyItem.get("role", None)
            contentValue = historyItem.get("content", None)

            if isinstance(roleValue, str) == True:
                if isinstance(contentValue, str) == True:
                    if roleValue == "user" or roleValue == "assistant":
                        messagesList.append({"role": roleValue, "content": contentValue})
        index = index + 1

    finalText, _ = agentRespond(userText, messagesList, toolDefinitions)
    return finalText

demo = gr.ChatInterface(
    fn=chatWithTools,
    title="FlyMeUP",
    type="messages"
)


demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://796ee03065bcb5f44b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
